# Structured‑Output Prompting & Validation
Generate **JSON‑only** responses, validate them, and auto‑retry on failures.

In [ ]:
!pip -q install openai ipywidgets

In [ ]:
import os, json, ipywidgets as w, time
from openai import OpenAI
from IPython.display import display, Markdown

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY', 'sk-'))

prompt_box = w.Textarea(value='Return a JSON with keys name, age, favorite_color.',
                        description='Prompt:',
                        layout=w.Layout(width='100%', height='70px'))
temperature = w.FloatSlider(value=0.7, min=0, max=2, step=0.1, description='Temp')
retries = w.IntSlider(value=3, min=1, max=5, step=1, description='Retries')
run_btn = w.Button(description='Run')
out = w.Output()

def call_model(prompt):
    r = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {'role':'system','content':'Respond ONLY with valid JSON.'},
            {'role':'user','content': prompt}
        ],
        temperature=temperature.value,
        max_tokens=128
    )
    return r.choices[0].message.content.strip()

def run(_):
    with out:
        out.clear_output()
        for i in range(1, retries.value+1):
            print(f'Attempt {i}')
            txt = call_model(prompt_box.value)
            try:
                data = json.loads(txt)
                display(Markdown('```json\n'+json.dumps(data,indent=2)+'\n```'))
                break
            except json.JSONDecodeError as e:
                print('Invalid JSON:', e)
                if i == retries.value:
                    print('All retries failed. Raw output:', txt)
                else:
                    time.sleep(1)
run_btn.on_click(run)
display(w.VBox([prompt_box, temperature, retries, run_btn, out]))